# DKUtils Documentation & Examples
This notebook gives a brief introduction to DKUtils and several examples using the main client that this package provides, DataKitchenClient. Multiple other clients and convenience functions & classes are provided by this package as described in the formal documention.

See the [DKUtils Documentation](https://datakitchen.github.io/DKUtils/) for a comprehensive overview of this [DKUtils Python Package](https://pypi.org/project/DKUtils/). Furthermore, the source code is publicly available on [GitHub DataKitchen/DKUtils](https://github.com/DataKitchen/DKUtils).

## Install DKUtils
Install the DKUtils package when you begin using it and anytime a newer version is available that you want to use.

In [ ]:
!pip install -U dkutils

## Imports and Global Variables
There are two ways to create a DataKitchenClient instance. The first is via its constructor by explicitly passing in a username and password. This method is typically used in General Purpose Container (GPC) python scripts:
```python
CLIENT = DataKitchenClient(username, password)
```
The second uses a DK CLI context in your home directory (see [DKCloudCommand Documentation](https://docs.datakitchen.io/articles/#!datakitchen-help/dkcloudcommand) for more information about the DK CLI).  This method is typically used for local development since it is simpler. In this example, a context is provided, but if you omit the context name, the default is used:
```python
CLIENT = create_using_context(context='Implementation')
```
In both instances, kitchen, recipe, and variation variables are optional. If any of these variables will be constant across all of your API calls, it's recommended to set them when instantiating the client:
```python
CLIENT = DataKitchenClient(username, password, kitchen=KITCHEN, recipe=RECIPE, variation=VARIATION)
CLIENT = create_using_context(context='Implementation', kitchen=KITCHEN, recipe=RECIPE, variation=VARIATION)
```


In [ ]:
import logging
import json

from datetime import datetime, timedelta
from pprint import pformat
from textwrap import indent

from dkutils.constants import INACTIVE_ORDER_STATUS_TYPES
from dkutils.datakitchen_api.datakitchen_client import DataKitchenClient, create_using_context

LOGGER = logging.getLogger()
LOGGER.addHandler(logging.StreamHandler())
LOGGER.setLevel(logging.INFO)

USERNAME = '[CHANGE_ME]'
PASSWORD = '[CHANGE_ME]'
CONTEXT  = '[CHANGE_ME]'

INDENT    = '    '
KITCHEN   = '[CHANGE_ME]'
RECIPE    = '[CHANGE_ME]'
VARIATION = '[CHANGE_ME]'

USE_CONTEXT = True
if USE_CONTEXT:
    CLIENT = create_using_context(context=CONTEXT, kitchen=KITCHEN, recipe=RECIPE, variation=VARIATION)
else:
    CLIENT = DataKitchenClient(USERNAME, PASSWORD, kitchen=KITCHEN, recipe=RECIPE, variation=VARIATION)

## Create an Order
If you didn't specify the kitchen, recipe, and variation when creating the DataKitchen Client, then you must set them before invoking `create_order`. Since we didn't set these fields above, we'll do that first here. There are two ways to do so. Since these attributes are properties with setters, one way is the following:
```python
CLIENT.kitchen = KITCHEN
CLIENT.recipe = RECIPE
CLIENT.variation = VARIATION
```
However, the client employs a somewhat [fluent interface](https://en.wikipedia.org/wiki/Fluent_interface) allowing method chaining for convenience:
```python
response = CLIENT.set_kitchen(KITCHEN).set_recipe(RECIPE).set_variation(VARIATION).create_order()
```

After setting the kitchen, recipe, and variation, simply invoke the `create_order` method. If the order takes parameters, an optional dictionary of parameters may be provided:
```python
parameters = {
    'parameter': 'value'
}
response = CLIENT.create_order(parameters=parameters)
```
If successful, the created order id is returned in the response and may be used to retrieve the order run id after it starts:
```python
order_id = response.json()['order_id']
```

In [ ]:
response = CLIENT.set_kitchen(KITCHEN).set_recipe(RECIPE).set_variation(VARIATION).create_order()
order_id = response.json()['order_id']
print(f'Created order id {order_id}')

## Retrieve Order Runs
Given an order id, retrieve the associated order runs. If an order was created as shown above, only a single order run should be retrieved. The order run id is captured in the `hid` field. If the order run has not yet started, the order runs array will be empty. Therefore, you will have to wait for the order run to start. Alternatively, there is a [create_and_monitor_orders](http://dkutils.dk.io/latest/docs/apidoc/dkutils.datakitchen_api.datakitchen_client.html#dkutils.datakitchen_api.datakitchen_client.DataKitchenClient.create_and_monitor_orders) method that can invoke multiple orders at once and wait for them all to finish. Please visit the provided link for details.

In [ ]:
order_runs = CLIENT.get_order_runs(order_id)
print(f'Order runs: \n{indent(pformat(order_runs), INDENT)}')

order_run_id = order_runs[0]['hid']
print(f'Order Run ID: {order_run_id}') 

## Retrieve Order Run Details

In [ ]:
order_run_details = CLIENT.get_order_run_details(order_run_id)
print(f'Order run details: \n{indent(pformat(order_run_details), INDENT)}')

## Retrieve Order Run Status

In [ ]:
order_run_status = CLIENT.get_order_run_status(order_run_id)
print(f'Order Run Status: {order_run_status}')

## Retrieve Kitchen Level Overrides
Retrieve all overrides for a kitchen.

In [ ]:
overrides = CLIENT.get_overrides()
print(f'Overrides:\n{indent(pformat(overrides), INDENT)}')

## Compare Kitchen Level Overrides
Compare kitchen overrides in your current kitchen with another kitchen. The `other` argument is optional - the default value is the parent kitchen.

In [ ]:
comparison = CLIENT.compare_overrides(other='IM_Production')
print(comparison)

## Add to Kitchen Staff
Simple method to add staff to a kitchen. Please note, you cannot modify the staff of a kitchen you're not a member of.

In [ ]:
new_kitchen_staff = ['[CHANGE_ME]']
CLIENT.set_kitchen(KITCHEN).add_kitchen_staff(new_kitchen_staff)